<a href="https://colab.research.google.com/github/ykitaguchi77/FundusPhoto/blob/main/Preprocess_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Preprocess_images**

In [8]:
from PIL import Image
import os
import time
from IPython.display import display
import numpy as np
import time
import os
import copy
import math
import shutil
import csv
import pandas as pd
import glob
from PIL import Image


#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**224pxに縮小して保存（Image.BILINEAR）**

In [9]:
path = '/content/drive/MyDrive/Deep_learning/FundusPhoto'
ext = 'png'  #png or jpg

os.chdir(path)


#元画像フォルダ
in_path = ['cropped_2_img', 'disc_2_img', 'macula_2_img', 'vascular_2_img']

#保存先フォルダ
out_path = ['cropped_224_img', 'disc_224_img', 'macula_224_img', 'vascular_224_img']


In [10]:
start_time = time.time()
process = [1]

for i in process:
    #保存先フォルダの作成
    if os.path.exists(out_path[i]):
        shutil.rmtree(out_path[i])
    os.makedirs(out_path[i])

    k=0
    for j in os.listdir(in_path[i]):
        path = os.path.join(in_path[i], str(j))
        print(path)
        pilr_img = Image.open(path).convert('RGB')
        converted_img = pilr_img.resize((224, 224), Image.BILINEAR)
        converted_img.save(os.path.join(out_path[i], str(j)))
        print(str(k)+' images converted!')
        k+=1
    
    
    # calculate elapsed time
    elapsed_time = int(time.time() - start_time)

    # convert second to hour, minute and seconds
    elapsed_hour = elapsed_time // 3600
    elapsed_minute = (elapsed_time % 3600) // 60
    elapsed_second = (elapsed_time % 3600 % 60)

    # print as 00:00:00
    print(str(elapsed_hour).zfill(2) + ":" + str(elapsed_minute).zfill(2) + ":" + str(elapsed_second).zfill(2))


disc_2_img/img52046426_02_2L.png
0 images converted!
disc_2_img/img13557213_01_1R.png
1 images converted!
disc_2_img/img73830255_10_1R.png
2 images converted!
disc_2_img/img54920602_00_1R.png
3 images converted!
disc_2_img/img75705547_00_1R.png
4 images converted!
disc_2_img/img52066364_06_1R.png
5 images converted!
disc_2_img/img29688883_04_1R.png
6 images converted!
disc_2_img/img41268862_02_2L.png
7 images converted!
disc_2_img/img28278415_03_2L.png
8 images converted!
disc_2_img/img70020805_00_1R.png
9 images converted!
disc_2_img/img10117692_01_2R.png
10 images converted!
disc_2_img/img25242385_01_2L.png
11 images converted!
disc_2_img/img48321148_01_2L.png
12 images converted!
disc_2_img/img06457849_01_1R.png
13 images converted!
disc_2_img/img67762668_09_1R.png
14 images converted!
disc_2_img/img37075143_01_1R.png
15 images converted!
disc_2_img/img40263861_05_2L.png
16 images converted!
disc_2_img/img29040083_07_2L.png
17 images converted!
disc_2_img/img41425244_00_2L.png
18 im

#**Split dataset for crossvalidation**

In [16]:
def get_path(orig_path, dst_path, split_num):
    #データの分割数を設定
    data_list = [0]
    k=0
    data_list = glob.glob(orig_path+'/*')
    split_length = int(len(data_list)/split_num)
    return data_list, split_length

def makefolder(dst_path):
    #フォルダを作成
    if os.path.exists(dst_path): #もしすでにフォルダがあれば削除
        shutil.rmtree(dst_path)
    os.mkdir(dst_path)  # 作成したいフォルダ名を作成
    for i in range(split_num):
        os.mkdir(dst_path+'/'+str(i))
        os.mkdir(dst_path+'/'+str(i)+'/train')
        os.mkdir(dst_path+'/'+str(i)+'/val')

def split_data_list(data_list, split_num):
    split_data, dst_data, dst_train, dst_val, dst_test = [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num

    #データの分割
    split_data = list(np.array_split(data_list, split_num))

    #データセット全体と分割したデータの差分を取り、dst_dataに格納

    dst_data = [0] * split_num
    for i in range(split_num):
        dst_data[i] = [x for x in data_list if x not in split_data[i]]

    #トレーニングセット、バリデーションセット、テストセットのリスト作成
    for i in range(split_num):
        dst_train[i] = dst_data[i]
        dst_val[i] = split_data[i]  #テストセット
    
    return dst_train, dst_val

def copy_to_folders(split_num,  dst_train, dst_val, dst_path):
    k=0
    for i in range(split_num):
        dst_path_train = dst_path +'/'+str(i)+'/train'
        dst_path_val = dst_path +'/'+str(i)+'/val'
        for p in dst_train[k]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_train)
            #print(p)
            print(dst_path_train)

        for p in dst_val[k]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_val)
            #print(p)    
            print(dst_path_val)

        k+=1

In [ ]:
out_list = ["cropped_224", "disc_224", "macula_224", "vascular_224"]

for i in out_list:
    orig_path = "/content/drive/MyDrive/Deep_learning/FundusPhoto/"+str(i)+"_img"
    dst_path = "/content/drive/MyDrive/Deep_learning/FundusPhoto/"+str(i)+"_img_trainval"  # フォルダ名
    split_num = 5  #データをいくつに分割するかを記載

    data_list, split_length = get_path(orig_path, dst_path, split_num)
    makefolder(dst_path)

    dst_train, dst_val = split_data_list(data_list, split_num)
    copy_to_folders(split_num, dst_train, dst_val, dst_path)

/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_224_img_trainval/0/train
/content/drive/M